In [ ]:
!python treshold_mine.py

In [12]:
import os
import numpy as np
from PIL import Image
from astropy.io import fits
import matplotlib.pyplot as plt

# Paths
FITS_DIR = '../data/new_data_subset/cut_fits'
MASKS_DIR = '../data/new_data_subset/masks_from_cut_fits/cv_masks'
OUTPUT_DIR = '../data/new_data_subset/masks_from_cut_fits/cut_fits_masked_cv'

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

for mask_filename in os.listdir(MASKS_DIR):
    fits_filename = mask_filename.replace('.png', '.fits')
    fits_path = os.path.join(FITS_DIR, fits_filename)

    if not os.path.exists(fits_path):
        print(f"[!] No matching FITS file for {mask_filename}. Skipping.")
        continue

    # Open FITS file
    with fits.open(fits_path) as hdul:
        data = hdul[0].data
        header = hdul[0].header

    # Load mask (expecting exact size match with FITS data)
    mask_path = os.path.join(MASKS_DIR, mask_filename)
    mask_image = Image.open(mask_path).convert("L")
    mask_array = np.array(mask_image)

    # Sanity check: assert exact size match
    if mask_array.shape != data.shape:
        print(f"[!] Size mismatch: mask {mask_array.shape} vs FITS {data.shape}")
        continue

    # Ensure binary (0 or 255)
    mask_array = (mask_array > 128).astype(np.uint8) * 255

    # Flip vertically to match FITS bottom-up orientation
    mask_array = mask_array[::-1]

    # Apply mask to FITS data (mask 0 = background, 255 = keep original data)
    bin_mask = (mask_array == 0)
    new_image = np.copy(data)
    new_image[bin_mask] = 0

    # Save masked FITS
    output_fits_path = os.path.join(OUTPUT_DIR, fits_filename)
    fits.writeto(output_fits_path, new_image, header=header, overwrite=True)

[!] No matching FITS file for Thumbs.db. Skipping.
